<a href="https://colab.research.google.com/github/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [1]:
from run import *
from agent import AgentDQN, AgentUADQN
from StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf

# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [2]:
# Agent
args = Arguments(if_on_policy=False)
args.agent = AgentUADQN()

# Environment
tickers = 'INO'  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1
initial_capital = 1e6
initial_stocks = 100
buy_cost_pct = 1e-3
sell_cost_pct = 1e-3
start_date = '2008-03-19'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False, if_save=True)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True, if_save=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = 2000
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = 2000
args.batch_size = 32
args.repeat_times = 4 # repeat_times * target_step == number of times we update before training
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = True
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)
args.target_step = 50 # number of exploration steps before training

| FeatureEngineer: start processing data (2 minutes)
| YahooDownloader: start downloading data (1 minute)
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (5284, 8)
| YahooDownloader: finish downloading data
Successfully added technical indicators
Successfully added turbulence index
| FeatureEngineer: finish processing data
| price_ary.shape: (1962,), tech_ary.shape: (1962, 8), turbulence_ary.shape: (1962,)
| price_ary.shape: (1259,), tech_ary.shape: (1259, 8), turbulence_ary.shape: (1259,)


# **Part 6: Train and Evaluate the Agent**

> The training and evaluating processes are all finished inside function **train_and_evaluate_mp()**, and the only parameter for it is **args**. It includes the fundamental objects in DRL:

*   agent,
*   environment.

> And it also includes the parameters for training-control:

*   batch_size,
*   target_step,
*   reward_scale,
*   gamma, etc.

> The parameters for evaluation-control:

*   break_step,
*   random_seed, etc.






In [3]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentUADQN/StockTradingEnv-v1_0
| Remove history
ID      Step      MaxR |    avgR      stdR       avgQ      avgL |  avgS  stdS
start exploring before training
Begin updating network before training
start training
explore environment
update environment: total_steps=100
0   5.00e+01      1.00 |
0   5.00e+01      1.00 |1.002698  0.000000   0.008088  0.254210 |  1258     0
explore environment
update environment: total_steps=150
0   1.00e+02      1.00 |0.999829  0.000000   0.008802  0.263615 |  1258     0
explore environment
update environment: total_steps=200
explore environment
update environment: total_steps=250
explore environment
update environment: total_steps=300
explore environment
update environment: total_steps=350
explore environment
update environment: total_steps=400
0   3.50e+02      1.00 |1.000213  0.000000   0.008842  0.255338 |  1258     0
explore environment
update environment: total_steps=450
explore environment
update environment: total_steps=500
expl

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [4]:
args.env_eval.draw_cumulative_return(args, torch)

Loaded act: ./AgentUADQN/StockTradingEnv-v1_0
The initial captial is 1000000.0
The initial number of stocks is 100


[0.9999902406194786,
 0.9999722833593193,
 0.999935197713338,
 0.9999363745970351,
 0.9998949806530041,
 0.9999140989593378,
 0.9998763079990225,
 0.9998750951851901,
 0.9998804290145603,
 0.9998498129009642,
 0.9998569188113231,
 0.9998645127609755,
 0.9998657892879478,
 0.999873448449781,
 0.9998840861745493,
 0.9998985534802342,
 0.9999432319242609,
 1.0000411056675456,
 1.000046694759369,
 1.0000211355819988,
 0.9999695496280114,
 0.9999946999537015,
 0.9999662121697133,
 0.999964894793878,
 0.9999466544921648,
 0.9999399262971583,
 0.9999874801865056,
 0.9999966212321959,
 1.000032097290874,
 1.0000739133944765,
 1.0000172244673136,
 1.0000172309124085,
 1.0000448761534613,
 1.0000160689956974,
 1.0000170279563312,
 1.0000208757248292,
 1.0000233025470422,
 0.9999984224234651,
 1.0000333519646414,
 1.0000829365819839,
 1.000091937861634,
 1.0000869090168087,
 1.000149337490309,
 1.000073133256924,
 1.000088872393367,
 1.000046944982078,
 1.0000706554399958,
 1.0000441829641815,
 1

In [11]:
args.env_eval.draw_cumulative_return_while_learning(args, torch)

Loaded act: ./AgentUADQN/StockTradingEnv-v1_0
updating network: 49
updating network: 99
updating network: 149
updating network: 199
updating network: 249
updating network: 299
updating network: 349
updating network: 399
updating network: 449
updating network: 499
updating network: 549
updating network: 599
updating network: 649
updating network: 699
updating network: 749
updating network: 799
updating network: 849
updating network: 899
updating network: 949
updating network: 999
updating network: 1049
updating network: 1099
updating network: 1149
updating network: 1199
updating network: 1249


[1.0,
 1.0,
 1.00000000575,
 1.0000000215102292,
 0.9999991961595808,
 0.9999993916996379,
 0.9999986568698668,
 0.9999986268691802,
 0.9999987820797525,
 0.9999976171599049,
 0.9999978221689896,
 0.9999979571093329,
 0.9999979671095618,
 0.9999980422492565,
 0.9999982422490657,
 0.9999985076192945,
 0.9999997376202673,
 1.000002437619123,
 1.000002580889695,
 1.000002120889657,
 1.0000011948495806,
 1.000001421109848,
 1.000000921109848,
 1.0000009071093903,
 1.0000004329498482,
 1.0000001871700768,
 1.000001417169619,
 1.0000016571693902,
 1.0000025505999621,
 1.0000032575197715,
 1.0000018475189705,
 1.0000018475189705,
 1.0000025441990275,
 1.000001590639943,
 1.0000016341887035,
 1.0000017606695426,
 1.0000018671694473,
 1.000000673470592,
 1.0000026400487227,
 1.000005753077655,
 1.0000062501188565,
 1.0000059371191616,
 1.0000102645993332,
 1.0000044715000964,
 1.0000057844788375,
 1.0000025911771206,
 1.0000045711804777,
 1.0000023644973306,
 1.0000009644986658,
 1.000001357918